In [8]:
import numpy as np
import pandas as pd

# Read the CSV file and parse dates in the first column
file_path = 'data.csv'
dta = pd.read_csv(file_path, parse_dates=[0])

# Print the first five rows of the dataframe
print(dta.head())


  Unnamed: 0  year  Cnsmr  Manuf  HiTec   Hlth  Other  SPCSPI    IR  \
0          1  1927  26.40  31.78  37.88  56.54  37.34   13.40  4.26   
1          2  1928  38.18  51.25  59.14  32.58  30.57   17.53  4.64   
2          3  1929 -43.01 -27.61  -6.18 -17.51 -22.02   24.86  6.01   
3          4  1930 -35.04 -39.94 -39.83 -17.87 -39.68   21.71  4.15   
4          5  1931 -35.16 -48.58 -48.64 -37.21 -53.03   15.98  2.43   

   pc_consumption  
0     5760.196756  
1     5818.535581  
2     6071.470984  
3     5686.178628  
4     5469.072499  


In [11]:
dta['c_growth'] = dta['pc_consumption'] / dta['pc_consumption'].shift(1)
dta['c_growth_lag1'] = dta['c_growth'].shift(1)
dta['r_lag1'] = dta['IR'].shift(1)
dta['r_lag2'] = dta['IR'].shift(2)
dta['r_forw1'] = dta['IR'].shift(-1)
dta['c_lag1'] = dta['pc_consumption'].shift(1)
dta['c_forw1'] = dta['pc_consumption'].shift(-1)
dta['const'] = 1

dta_clean = dta.dropna()


endog_df = dta_clean[['r_forw1', 'c_forw1', 'pc_consumption']]
exog_df = endog_df
instrument_df = dta_clean[['r_lag1', 'r_lag2', 'c_growth', 'c_growth_lag1',
                           'const']]

endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])
print(dta.head())

  Unnamed: 0  year  Cnsmr  Manuf  HiTec   Hlth  Other  SPCSPI    IR  \
0          1  1927  26.40  31.78  37.88  56.54  37.34   13.40  4.26   
1          2  1928  38.18  51.25  59.14  32.58  30.57   17.53  4.64   
2          3  1929 -43.01 -27.61  -6.18 -17.51 -22.02   24.86  6.01   
3          4  1930 -35.04 -39.94 -39.83 -17.87 -39.68   21.71  4.15   
4          5  1931 -35.16 -48.58 -48.64 -37.21 -53.03   15.98  2.43   

   pc_consumption  c_growth  c_growth_lag1  r_lag1  r_lag2  r_forw1  \
0     5760.196756       NaN            NaN     NaN     NaN     4.64   
1     5818.535581  1.010128            NaN    4.26     NaN     6.01   
2     6071.470984  1.043471       1.010128    4.64    4.26     4.15   
3     5686.178628  0.936541       1.043471    6.01    4.64     2.43   
4     5469.072499  0.961819       0.936541    4.15    6.01     3.36   

        c_lag1      c_forw1  const  
0          NaN  5818.535581      1  
1  5760.196756  6071.470984      1  
2  5818.535581  5686.178628      1 

In [12]:
def moment_consumption1(params, exog):
    beta, gamma = params
    r_forw1, c_forw1, c = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = 1 - beta * (1 + r_forw1) * np.power(c_forw1 / c, -gamma)
    return -err

In [13]:
endog1 = np.zeros(exog.shape[0])    
mod1 = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4)
w0inv = np.dot(instrument.T, instrument) / len(endog1)
res1 = mod1.fit([1,-1], maxiter=2, inv_weights=w0inv) 

Optimization terminated successfully.
         Current function value: 0.137095
         Iterations: 17
         Function evaluations: 23
         Gradient evaluations: 23
Optimization terminated successfully.
         Current function value: 0.217435
         Iterations: 8
         Function evaluations: 14
         Gradient evaluations: 14


In [14]:
print(res1.summary(yname='Euler Eq', xname=['discount', 'CRRA']))


                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                        17.39
Model:                 NonlinearIVGMM   Prob (Hansen J):              0.000167
Method:                           GMM                                         
Date:                Tue, 04 Apr 2023                                         
Time:                        21:19:59                                         
No. Observations:                  80                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
discount       0.2048      0.023      8.795      0.000       0.159       0.250
CRRA          20.3853      6.578      3.099      0.002       7.493      33.277


In [15]:

# We don't need Nelder-Mead in this case, we can use bfgs default directly
#res1_ = mod1.fit([1,-1], maxiter=0, inv_weights=w0inv, opt_method='nm')

res1_hac4_2s = mod1.fit([1, -1], maxiter=2, inv_weights=w0inv, weights_method='hac', wargs={'maxlag':4})
print('\n\n')
print(res1_hac4_2s.summary(yname='Euler Eq', xname=['discount', 'CRRA']))

Optimization terminated successfully.
         Current function value: 0.137095
         Iterations: 17
         Function evaluations: 23
         Gradient evaluations: 23
Optimization terminated successfully.
         Current function value: 0.065243
         Iterations: 8
         Function evaluations: 13
         Gradient evaluations: 13



                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                        5.219
Model:                 NonlinearIVGMM   Prob (Hansen J):                0.0736
Method:                           GMM                                         
Date:                Tue, 04 Apr 2023                                         
Time:                        21:21:30                                         
No. Observations:                  80                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------